# CFEM - Itabira

## 1. Coleta de Dados

### 1.1 Conexão

In [1]:
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import json
from bs4 import BeautifulSoup
import os


driver = webdriver.Chrome()
url_paginaInicial = "https://servicositabira.govbr.cloud/pronimtb/"
url_despesas_fonteDeRecurso = "https://servicositabira.govbr.cloud/pronimtb/index.asp?acao=3&item=6"

driver.get(url_despesas_fonteDeRecurso) # Pagina Inicial > Despesas > Fonte de Recurso

# Esperar até que o DOM esteja carregado
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))


<selenium.webdriver.remote.webelement.WebElement (session="4d0e7b784a14f2e06072bb1249d61271", element="f.7F326916E6FF8A8B4E641505607FE2FB.d.DAAB09633C1541E304C9B931A7AB431D.e.250")>

### 1.2 Filtros

In [2]:
# Data inicial
data_inicial = driver.find_element(By.ID, "txtDataInicial")
data_inicial.clear()
data_inicial.send_keys("01012024")

# Data final 
data_final = driver.find_element(By.ID, "txtDataFinal")
data_final.clear()
data_final.send_keys("31012024")


# Unidade Gestora
select_element = driver.find_element(By.ID, 'cmbUnidadeGestora')
select = Select(select_element)
select.select_by_value('0')  # Selecionar a opção com valor '0'

# Clicar no botão "Gerar"
element = driver.find_element(By.ID, "confirma")
element.click()

### 1.3 Função ler_tabela()

In [75]:
# def ler_tabela(nome_dados):

#     html_table = wait.until(EC.presence_of_element_located((By.ID, 'tbTabela')))

#     # Passar o objeto html_table para BeautifulSoup
#     soup = BeautifulSoup(html_table.get_attribute('outerHTML'), 'html.parser')

#     # Find the table element
#     table = soup.find('table')

#     # Initialize an empty dictionary to store the table data
#     table_data = {}

#     # Extract the headers from the table
#     header_row = table.find_all('tr')[1]  # Assuming the second row is the header
#     headers = header_row.find_all('th')
#     headers_text = [header.text.strip() for header in headers]

#     # Extract the data from the table
#     for row in table.find_all('tr')[2:]:  # Starting from the third row to skip the header and total row
#         cells = row.find_all('td')
#         item = cells[0].get_text().strip()
#         data = {
#             headers_text[1]: cells[1].get_text().strip(),
#             headers_text[2]: cells[2].get_text().strip(),
#             headers_text[3]: cells[3].get_text().strip(),
#             headers_text[4]: cells[4].get_text().strip()
#         }
#         table_data[item] = data

    

#     # Convert the data to a JSON string with indentation
#     json_data = json.dumps(table_data, indent=4, ensure_ascii=False)

#     # Print the indented JSON
#     #print(json_data)

#     # Define the directory and file name
#     directory = 'json_output'
#     if not os.path.exists(directory):
#         os.makedirs(directory)
#     file_path = os.path.join(directory, f'{nome_dados}.json')

#     # Save the JSON data to a file
#     with open(file_path, 'w', encoding='utf-8') as json_file:
#         json_file.write(json_data)
#     print(f'Dados salvos em: {file_path}')

In [4]:
import json
import os
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

def ler_tabela(nome_dados):
    html_table = wait.until(EC.presence_of_element_located((By.ID, 'tbTabela')))
    
    # Passar o objeto html_table para BeautifulSoup
    soup = BeautifulSoup(html_table.get_attribute('outerHTML'), 'html.parser')
    
    # Find the table element
    table = soup.find('table')
    
    # Initialize an empty dictionary to store the table data
    table_data = {}
    
    # Extract the headers from the table
    header_row = table.find_all('tr')[1]  # Assuming the second row is the header
    headers = header_row.find_all('th')
    headers_text = [header.text.strip().replace('.', '') for header in headers]  # Remove dots from headers
    
    # Extract the data from the table
    for row in table.find_all('tr')[2:]:  # Starting from the third row to skip the header and total row
        cells = row.find_all('td')
        item = cells[0].get_text().strip().replace('.', '')  # Remove dots from item
        # Remove all dots from the text data
        cells_text = [cell.get_text().strip().replace('.', '') for cell in cells[1:]]
        data = {
            headers_text[i+1]: cells_text[i] for i in range(len(cells_text))
        }
        table_data[item] = data
    
    # Convert the data to a JSON string with indentation
    json_data = json.dumps(table_data, indent=4, ensure_ascii=False)
    
    # Define the directory and file name
    directory = 'json_output'
    if not os.path.exists(directory):
        os.makedirs(directory)
    file_path = os.path.join(directory, f'{nome_dados}.json')
    
    # Save the JSON data to a file
    with open(file_path, 'w', encoding='utf-8') as json_file:
        json_file.write(json_data)
    print(f'Dados salvos em: {file_path}')


### 1.4 Seleção de Recursos

1.4.1 CFEM

In [5]:
# Selecionando CFEM na Tabela # Pagina Inicial > Despesas > Fonte de Recurso > CFEM
wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
try:
    # Localize o link pelo texto
    link_cfem = driver.find_element(By.LINK_TEXT, "Transf União Referente à Compensação Financeira de Recursos Minerais")
    
    # Clique no link
    link_cfem.click()

except TimeoutException:
    print("Ocorreu um timeout ao esperar por um elemento.")
except NoSuchElementException:
    print("Não foi possível encontrar um elemento.")


1.4.2 Detalhamento da fonte de despesa (CFEM Ordinário)

In [6]:
ler_tabela("desp_por_fonte")

Dados salvos em: json_output\desp_por_fonte.json


In [8]:
# Selecionando Recurso Ordinário - CFEM na Tabela # Pagina Inicial > Despesas > Fonte de Recurso > CFEM > Recurso Ordinário
wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
try:
    # Localize o link pelo texto
    link_recurso_ord_cfem = driver.find_element(By.LINK_TEXT, "000.0911 - Recurso Ordinário - CFEM")
    
    # Clique no link
    link_recurso_ord_cfem.click()

except TimeoutException:
    print("Ocorreu um timeout ao esperar por um elemento.")
except NoSuchElementException:
    print("Não foi possível encontrar um elemento.")

### 2.1 Ler tabela e salvar links numa lista

In [9]:
ler_tabela("naturezas_despesas")

Dados salvos em: json_output\naturezas_despesas.json


In [81]:
desc_natureza_despesa:str = '4.4.90.51 - Obras e Instalações'
def seleciona_natureza_despesa(id_tabela, descrição_natureza):
  
  wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))


  table = driver.find_element(By.ID, id_tabela)

  # links = set()   ---> garante que os links sejam unicos
  links = []  # Lista garante a ordem de inserção dos links
  rows = table.find_elements(By.TAG_NAME, 'tr')
  for row in rows:
    try:
        link = row.find_element(By.XPATH, './td[1]//a') # encontrar um link dentro da primeira coluna
        href = link.get_attribute('href')
        links.append(href)  # Add para set / append para list
    except:
        continue
    #for link in links:
    #print(link)

  link_cfem_obras = driver.find_element(By.LINK_TEXT, descrição_natureza)
  link_cfem_obras.click()

    
seleciona_natureza_despesa('tbTabela',desc_natureza_despesa)

In [82]:
ler_tabela()

TypeError: ler_tabela() missing 1 required positional argument: 'nome_dados'

In [ ]:
def clicar_linhas_primeira_coluna(driver, tabela_id):
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    
    table = driver.find_element(By.ID, tabela_id)
    # Extrair os links da primeira coluna
    rows = table.find_elements(By.TAG_NAME, 'tr')
    
    for index, val in enumerate(rows):
        if index == 0 or index == 1: continue
        try:
            table = driver.find_element(By.ID, tabela_id)
            rows = table.find_elements(By.TAG_NAME, 'tr')
            link = rows[index].find_element(By.XPATH, './td[1]/a')
            link.click()
            ler_tabela()
            driver.back()
            WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.ID, tabela_id)))
            time.sleep(2)
        except Exception as err:  
            print(err)

clicar_linhas_primeira_coluna(driver,'tbTabela')

In [ ]:
import json
from bson.decimal128 import Decimal128

# Função para converter valores monetários
def converter_valores(json_obj, nome_arquivo):
    for key, value in json_obj.items():
        if isinstance(value, dict):
            converter_valores(value, nome_arquivo)  # Chamar recursivamente com o mesmo nome do arquivo
        else:
            if isinstance(value, str) and value.startswith("R$"):
                # Remover "R$ " e converter para float
                valor_convertido = value.replace("R$ ", "").replace(".", "").replace(",", ".")
                # Atualizar valor no JSON como Decimal128
                json_obj[key] = str(Decimal128(valor_convertido))




In [ ]:
# Nome do arquivo
nome_arquivo = "desp_por_fonte.json"

# Ler o arquivo JSON
with open(f"json_output/{nome_arquivo}", 'r', encoding='utf-8') as file:
    json_data = json.load(file)

# Converter os valores no JSON, passando o nome do arquivo como parâmetro
converter_valores(json_data, nome_arquivo)

# Salvar o JSON atualizado de volta no arquivo
with open(f"json_output/{nome_arquivo}", 'w', encoding='utf-8') as file:
    json.dump(json_data, file, ensure_ascii=False, indent=4)

print("Arquivo atualizado com sucesso.")

In [10]:
# Nome do arquivo
nome_arquivo = "naturezas_despesas.json"

# Ler o arquivo JSON
with open(f"json_output/{nome_arquivo}", 'r', encoding='utf-8') as file:
    json_data = json.load(file)

# Converter os valores no JSON, passando o nome do arquivo como parâmetro
converter_valores(json_data, nome_arquivo)

# Salvar o JSON atualizado de volta no arquivo
with open(f"json_output/{nome_arquivo}", 'w', encoding='utf-8') as file:
    json.dump(json_data, file, ensure_ascii=False, indent=4)

print("Arquivo atualizado com sucesso.")

Arquivo atualizado com sucesso.
